In [121]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle


In [115]:
win_counts = np.array([
    [0.0, 14.0, 29.5, 20.5, 3.5, 5.5],
    [754.0, 0.0, 94.5, 205.5, 115.5, 91.0],
    [738.5, 673.5, 0.0, 222.5, 199.0, 303.5],
    [747.5, 562.5, 545.5, 0.0, 396.5, 247.5],
    [764.5, 652.5, 569.0, 371.5, 0.0, 250.0],
    [762.5, 677.0, 464.5, 520.5, 518.0, 0.0]
])


path = "/Users/kartikpingle/Desktop/gorilla_ablation"

# this folder contains 4 folders: 0, 1, 2, 3
# 0 contains folders from 0 to 95
# 1 contains folders from 96 to 191
# 2 contains folders from 192 to 287
# 3 contains folders from 288 to 383
# all innermost folders have logs.txt files

for i in range(4):
    for j in range(96):
        with open(f"{path}/{i}/{i * 96 + j}/log.txt", "r") as f:
            with open("ALL_LOGS.txt", "a") as f2:
                f2.write(f.read())



In [116]:
def parse_game_log(file_path):
    game_logs = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove any leading/trailing whitespace
            parts = line.split()  # Split the line into parts
            
            if len(parts) == 3:  # Ensure the line is correctly formatted
                player_i = int(parts[0])  # First part: index of player i
                player_j = int(parts[1])  # Second part: index of player j
                result = int(parts[2])  # Third part: game result (-1, 0, 1)

                # Create a dictionary for this game log
                game_log = {
                    'player_i': player_i,
                    'player_j': player_j,
                    'result': result
                }

                # Append the dictionary to the list
                game_logs.append(game_log)

    return game_logs


In [117]:
def update_elo(ratings, player_i, player_j, result, k=32):
    # Calculate expected scores
    E_i = 1 / (1 + 10 ** ((ratings[player_j] - ratings[player_i]) / 400))
    E_j = 1 / (1 + 10 ** ((ratings[player_i] - ratings[player_j]) / 400))

   
    if result == 0:  # Player i wins
        S_i, S_j = 1, 0
    elif result == 1:  
        S_i, S_j = 0, 1
    else:  # Draw
        S_i, S_j = 0.5, 0.5

    # Update ratings
    ratings[player_i] += k * (S_i - E_i)
    ratings[player_j] += k * (S_j - E_j)

In [118]:
def compute_elo_from_logs(game_logs):

    # Initialize ratings (assuming no player indices beyond what's seen in the logs)
    ratings = {}
    for log in game_logs:
        ratings[log['player_i']] = 1500
        ratings[log['player_j']] = 1500
    np.random.seed(101)
    np.random.shuffle(game_logs)

    # Process each game
    for log in game_logs:
        update_elo(ratings, log['player_i'], log['player_j'], log['result'])

    return ratings


In [122]:
logs_file_oath = "ALL_LOGS.txt"

game_logs = parse_game_log(logs_file_oath)

elos = compute_elo_from_logs(game_logs)

for elo in elos:
    print(f"Bot {elo} has an ELO: {elos[elo]}")

# save elos to a pickle
with open('elos.pkl', 'wb') as f:
    pickle.dump(elos, f)



Bot 0 has an ELO: 1052.2069296280213
Bot 1 has an ELO: 1410.6355854118765
Bot 2 has an ELO: 1332.3077033617685
Bot 3 has an ELO: 1360.8330673685077
Bot 4 has an ELO: 1407.0138086517634
Bot 5 has an ELO: 1550.2207253043155
Bot 6 has an ELO: 1259.132270137405
Bot 7 has an ELO: 1489.1780552809691
Bot 8 has an ELO: 1560.0377678459859
Bot 9 has an ELO: 1379.2433444937942
Bot 10 has an ELO: 1435.843415796157
Bot 11 has an ELO: 1290.2803515686276
Bot 12 has an ELO: 1430.7334017807884
Bot 13 has an ELO: 1383.273028858832
Bot 14 has an ELO: 1373.4208436487972
Bot 15 has an ELO: 1393.3646631507445
Bot 16 has an ELO: 1474.5957937115513
Bot 17 has an ELO: 1393.800731305565
Bot 18 has an ELO: 1554.1789176089549
Bot 19 has an ELO: 1720.2500070472158
Bot 20 has an ELO: 1760.4385780592468
Bot 21 has an ELO: 1257.8800400838916
Bot 22 has an ELO: 1436.152481986947
Bot 23 has an ELO: 1472.5599448632152
Bot 24 has an ELO: 1636.028482993384
Bot 25 has an ELO: 1633.8374499180238
Bot 26 has an ELO: 1450.8385

In [120]:
# open the file, plug it into 